In [12]:
venue = {'01':'札幌','02':'函館','03':'福島','04':'新潟','05':'東京','06':'中山','07':'中京','08':'京都','09':'阪神','10':'小倉'}
# import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from urllib.request import urlopen
from Class.RaceFeature import RaceFeature
from Class.Results import Results
from Class.Peds import Peds
from Class.ShutubaTable import ShutubaTable

import matplotlib.pyplot as plt

import torch
import torch.optim as optim

import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
import random
import tqdm

from torch import nn

# pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', 200)
# np.set_printoptions(threshold=200)

In [13]:
import pandas as pd
import tqdm
import numpy as np
import time
from  urllib.request import urlopen

class Return:
    def __init__(self,return_tables):
        self.return_tables = return_tables
        # self.hukusho = self.return_tables[self.return_tables[0]=='複勝'][[1,2]]
    
    @classmethod
    def read_pickle(cls,path_list):
        df = pd.concat([pd.read_pickle(path) for path in path_list]) 
        return cls(df)
    
    @staticmethod
    def scrape(race_id_list,pre_df):
        return_tables = {}
        if pre_df is not None:
            pre_list=pre_df.index.unique()
            race_id_list = [race_id  for race_id in race_id_list if not race_id in pre_list]
        for race_id in tqdm.tqdm(race_id_list):
            try:
                url = 'https://db.netkeiba.com/race/'+race_id
                f = urlopen(url)
                html = f.read()
                html = html.replace(b'<br />', b'br')
                html = urlopen(url).read()
                dfs = pd.read_html(html)
                return_tables[race_id]  = pd.concat([dfs[1],dfs[2]])
                time.sleep(1)
            except IndexError:
                continue
            except KeyboardInterrupt:
                break
            except Exception as e:
                print(e)
                break
        
        for key in return_tables.keys():
            return_tables[key].index = [key]*len(return_tables[key])

        return_tables = pd.concat([return_tables[key] for key in return_tables.keys()],sort=False)
        df_return_tables = pd.DataFrame(return_tables)

        df_return_tables = pd.concat([pre_df,df_return_tables],axis=0)
        

        return df_return_tables
            
    @property
    def hukusho(self):
        hukusho = self.return_tables[self.return_tables[0]=='複勝'][[1,2]]
        wins = hukusho[1].str.split(' ',expand=True)[[0,1,2]]
        wins.columns = ['win_0','win_1','win_2']
        returns = hukusho[2].str.split(' ',expand=True)[[0,1,2]]
        returns.columns = ['return_0','return_1','return_2']
        df = pd.concat([wins,returns],axis=1)
        for column in df.columns:
            df[column] = df[column].str.replace(',','')
        return df.fillna(0).astype(int) 

    @property
    def tansho(self):
        tansho = self.return_tables[self.return_tables[0]=='単勝'][[1,2]]
        tansho.columns = ['win','return']

        for column in tansho.columns:
            tansho[column] = pd.to_numeric(tansho[column],errors='coerce')

        return tansho


In [14]:
# return_tables = {}
# pre_df=None
# race_id_list = r.data.index.unique()
# race_id_list = race_id_list[:3]
# if pre_df is not None:
#     pre_list=pre_df.index.unique()
#     race_id_list = [race_id  for race_id in race_id_list if not race_id in pre_list]
# for race_id in tqdm.tqdm(race_id_list):
#     try:
#         url = 'https://db.netkeiba.com/race/'+race_id
#         f = urlopen(url)
#         html = f.read()
#         html = html.replace(b'<br />', b'br')
#         html = urlopen(url).read()
#         dfs = pd.read_html(html)
#         return_tables[race_id]  = pd.concat([dfs[1],dfs[2]])
#         time.sleep(1)
#     except IndexError:
#         print('tetete')
#         continue
#     except Exception as e:
#         print('tetete')
#         print(e)
#         break

# for key in return_tables.keys():
#     return_tables[key].index = [key]*len(return_tables[key])
# print('tetete')
# return_tables = pd.concat([return_tables[key] for key in return_tables.keys()],sort=False)
# df_return_tables = pd.DataFrame(return_tables)

# df_return_tables = pd.concat([pre_df,df_return_tables],axis=0)

In [15]:
r = Results.read_pickle(['pickle/results.pickle'])
p = Peds.read_pickle(['pickle/peds.pickle'])
rt = Return.read_pickle(['pickle/return_tables.pickle'])

In [16]:
# def get_return_info(data,rt):
#     df1 = pd.merge(
#         data.reset_index(),
#         rt.tansho.reset_index().rename(columns={'win': '馬番'})[['index', 'return', '馬番']],
#         on=['index', '馬番'],
#         how='left'
#     ).fillna(0)

#     # 複勝データ (rt.hukusho) を一度に縦持ちに変換
#     hukusho_merged = pd.wide_to_long(
#         rt.hukusho.reset_index(),
#         stubnames=['win', 'return'],
#         i='index',
#         j='num',
#         sep='_'
#     ).reset_index().rename(columns={'win': '馬番', 'return': 'hukusho_return'})

#     # 単勝データと複勝データをマージし、複勝リターンを結合
#     df1 = pd.merge(df1, hukusho_merged[['index', '馬番', 'hukusho_return']], on=['index', '馬番'], how='left')

#     df1 = df1.fillna(0).set_index('index')
#     df1 = df1.rename(columns={'return': 'tansho_return'})
#     df1['tansho_return'] = df1['tansho_return']*0.01
#     df1['hukusho_return'] = df1['hukusho_return']*0.01
#     return df1


In [17]:
r.preprocessing()
p.get_no_peds_list(r.data_p)
p.encode()
r.get_previous_result()
# r.merge_peds(p.peds_e)
r.data_pe = r.data_p
r.process_categorical()
r.get_return_info(rt)

0it [00:00, ?it/s]


In [18]:
import pickle
with open("pickle/pickle_r_all", "wb") as f:
    pickle.dump(r, f)

with open("pickle/pickle_p_all", "wb") as f:
    pickle.dump(p, f)


# with open("pickle/pickle_r_all", "rb") as f:
#     r = pickle.load(f)
# with open("pickle/pickle_p_all", "rb") as f:
#     p = pickle.load(f)

In [19]:
# # shutuba_table = ShutubaTable.scrape_shutuba_table_ChromeDriver(['202404030810','202404030811'],'2024-05-01')
# shutuba_table = pd.read_pickle('pickle/shutuba_table_sample.pickle')
# st = ShutubaTable(shutuba_table)
# st.data_p = shutuba_table
# p.get_no_peds_list(st.data_p)
# p.encode()
# st.read_encode_data(r)
# st.merge_peds(p.peds_e)
# st.process_categorical()
# st.data_c = pd.concat([st.data_c,r.data_c])
# st.data_c = st.data_c.fillna(0)

In [20]:
# import pickle
# with open("pickle/pickle_r_all", "wb") as f:
#     pickle.dump(r, f)

# with open("pickle/pickle_p_all", "wb") as f:
#     pickle.dump(p, f)


# with open("pickle/pickle_r_all", "rb") as f:
#     hoge = pickle.load(f)

In [21]:
# pred_horse_list = st.data['horse_id'].unique()
# for i in range(4):
#     pred_index_list = r.data[r.data['horse_id'].isin(pred_horse_list)].index.unique()
#     pred_horse_list = r.data[r.data.index.isin(pred_index_list)]['horse_id'].unique()
# data =r.data_c[ r.data.index.isin(pred_index_list)]